# Collect Demonstration from Keyboard

Collect demonstration data for the given environment.
The task is to pick a mug and place it on the plate. The environment recognizes the success if the mug is on the plate, gthe ripper opened, and the end-effector positioned above the mug.

<img src="./media/teleop.gif" width="480" height="360">

Use WASD for the xy plane, RF for the z-axis, QE for tilt, and ARROWs for the rest of rthe otations. 

SPACEBAR will change your gripper's state, and Z key will reset your environment with discarding the current episode data.

For overlayed images, 
- Top Right: Agent View 
- Bottom Right: Egocentric View
- Top Left: Left Side View
- Bottom Left: Top View

In [1]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.so100_env import SO100Env
# from mujoco_env.y_env2_block import SimpleEnv2Block
from lerobot.datasets.lerobot_dataset import LeRobotDataset

/home/dragon/anaconda3/envs/mujoco_vla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
SEED = 0 
# SEED = None <- Uncomment this line to randomize the object positions

REPO_NAME = 'omy_pnp'
NUM_DEMO = 1 # Number of demonstrations to collect
ROOT = "./demo_data" # The root directory to save the demonstrations

In [3]:
TASK_NAME = 'Put mug cup on the plate' 
# xml_path = './asset/example_scene_y_so100.xml'
xml_path = './asset/scene_block_so100.xml'
# Define the environment
PnPEnv = SO100Env(xml_path, seed = SEED, state_type = 'joint_angle')
# PnPEnv = SimpleEnv2Block(xml_path, seed = SEED, state_type = 'joint_angle')
# 启动teleoptable的窗口以后，不用管中断的问题，直接运行下一步的cell


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[27] n_qvel:[24] n_qacc:[24] n_ctrl:[6]
 integrator:[RK4]

n_body:[18]
 [0/18] [world] mass:[0.00]kg
 [1/18] [front_object_table] mass:[1.00]kg
 [2/18] [camera] mass:[0.00]kg
 [3/18] [camera2] mass:[0.00]kg
 [4/18] [camera3] mass:[0.00]kg
 [5/18] [Base] mass:[0.56]kg
 [6/18] [Rotation_Pitch] mass:[0.12]kg
 [7/18] [Upper_Arm] mass:[0.16]kg
 [8/18] [Lower_Arm] mass:[0.15]kg
 [9/18] [Wrist_Pitch_Roll] mass:[0.07]kg
 [10/18] [Fixed_Jaw] mass:[0.09]kg
 [11/18] [camera_center] mass:[0.00]kg
 [12/18] [Moving_Jaw] mass:[0.02]kg
 [13/18] [body_obj_plate_11] mass:[0.00]kg
 [14/18] [object_plate_11] mass:[0.10]kg
 [15/18] [body_obj_block_red] mass:[0.30]kg
 [16/18] [body_obj_mug_5] mass:[0.00]kg
 [17/18] [object_mug_5] mass:[0.08]kg
body_total_mass:[2.66]kg

n_geom:[101]
geom_names:['floor', 'front_object_table', None, None, None, None, None, None, None, None, None, None, Non

## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [4]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (6,),
                        # "shape": (9,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
                tolerance_s=0.1
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Directory ./demo_data already exists.


## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [5]:
import time
import numpy as np
from PIL import Image

action = np.zeros(7)
episode_id = 0
record_flag = False
episode_start_time = None

while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        current_time = time.time()
        
        # 检查成功
        done = PnPEnv.check_success()
        if done and record_flag: 
            # 保存episode - 不需要任何参数
            dataset.save_episode()  # ✅ 正确的调用方式
            PnPEnv.reset(seed=SEED)
            episode_id += 1
            record_flag = False
            episode_start_time = None
            print(f"✅ Episode {episode_id} completed and saved")
            
        # 获取动作
        action, reset = PnPEnv.teleop_robot()
        
        # 开始记录逻辑
        if not record_flag and np.any(np.abs(action) > 0.01):
            record_flag = True
            episode_start_time = current_time
            print("🔴 Start recording")
            
        if reset:
            PnPEnv.reset(seed=SEED)
            dataset.clear_episode_buffer()  # 清空缓冲区
            record_flag = False
            episode_start_time = None
            print("🔄 Environment reset")
            
        # 获取状态和图像
        ee_pose = PnPEnv.get_ee_pose()
        agent_image, wrist_image = PnPEnv.grab_image()
        
        # 调整图像大小
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        
        # 执行动作
        joint_q = PnPEnv.step(action)
        
        if record_flag:
            # 计算相对时间戳
            relative_timestamp = current_time - episode_start_time
            
            # 添加帧数据
            dataset.add_frame(
                frame={
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": ee_pose.astype(np.float32),
                    "action": joint_q.astype(np.float32),
                    "obj_init": PnPEnv.obj_init_pose.astype(np.float32),
                },
                task=TASK_NAME,
                timestamp=float(relative_timestamp)
            )
            
        # 渲染
        PnPEnv.render(teleop=True)

print(f"🎉 Recording completed! Total episodes collected: {episode_id}")


🔴 Start recording
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
DONE INITIALIZATION
🔄 Environment reset
🔴 Start recording
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
DONE INITIALIZATION
🔄 Environment reset
🔴 Start recording
🎉 Recording completed! Total episodes collected: 0


In [6]:
PnPEnv.env.close_viewer()

In [7]:
# Clean up the images folder
import shutil
# shutil.rmtree(dataset.root / 'images')